In [68]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [69]:
#set up executable path
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Eunjeong\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


### Scrape News from the website

In [70]:
#setup beautiful soup for redplanetscience.com

#URL to be scraped
url = "https://www.redplanetscience.com"

#visit url
browser.visit(url)

#Grab the html 
news_html = browser.html

#create beautiful soup object with parser - parse in the html elements
soup = BeautifulSoup(news_html, "html.parser")

In [71]:
#find most recent news title
news_title = soup.find('div', class_="content_title").text
news_title

"All About the Laser (and Microphone) Atop Mars 2020, NASA's Next Rover"

In [72]:
#find paragraph text
news_p = soup.find('div', class_="article_teaser_body").text
news_p

'SuperCam is a rock-vaporizing instrument that will help scientists hunt for Mars fossils.'

In [73]:
#Make a dictionary with the values
redplanet_dict = {"news_title": "NASA's Perseverance Rover 100 Days Out", "news_p": "Mark your calendars: The agency's latest rover has only about 8,640,000 seconds to go before it touches down on the Red Planet, becoming history's next Mars car."}
redplanet_dict

{'news_title': "NASA's Perseverance Rover 100 Days Out",
 'news_p': "Mark your calendars: The agency's latest rover has only about 8,640,000 seconds to go before it touches down on the Red Planet, becoming history's next Mars car."}

### Scrape photo from mars space image website

In [74]:
#set up url for. the browser to visit using splinter
url="https://spaceimages-mars.com/"
html = browser.visit(url)

In [75]:
#locate the button and click it
image_element = browser.find_by_tag("button")[1]
image_element.click()

In [76]:
#Parse with beautiful soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [77]:
#Select image src and add it to the url
image_anchor = img_soup.find("img", class_="fancybox-image").get('src')
featured_image_url = url + image_anchor
print(featured_image_url)
print(image_anchor)

https://spaceimages-mars.com/image/featured/mars3.jpg
image/featured/mars3.jpg


In [78]:
#Make a dictionary of space image
space_image_dict = {"featured_image": "https://spaceimages-mars.com/image/featured/mars3.jpg"}
space_image_dict

{'featured_image': 'https://spaceimages-mars.com/image/featured/mars3.jpg'}

### Mars Facts

In [79]:
mars_table = pd.read_html('https://galaxyfacts-mars.com')[0]
print(mars_table)

                         0                1                2
0  Mars - Earth Comparison             Mars            Earth
1                Diameter:         6,779 km        12,742 km
2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
3                   Moons:                2                1
4       Distance from Sun:   227,943,824 km   149,598,262 km
5          Length of Year:   687 Earth days      365.24 days
6             Temperature:     -87 to -5 °C      -88 to 58°C


In [80]:
mars_table.columns = ['description', 'mars', 'earth']
mars_table.set_index('description', inplace=True)
mars_table

,mars,earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


### Scrape hemisphere data from mars hemispheres

In [81]:
#Visit mars hemispheres website
url = "https://www.marshemispheres.com"
browser.visit(url)

In [82]:
#List which will hold the dictionaries
hemisphere_image_links = []


#get links for all hemispheres
links = browser.find_by_css('a.product-item img')

#click all links and get the href
for i in range(len(links)):
    hemisphere_dict = {}
    browser.find_by_css('a.product-item img')[i].click()
    thumb_img = browser.links.find_by_text('Sample').first
    print(thumb_img)
    
    #hemisphere href
    hemisphere_dict['img_url'] = thumb_img['href']
    
    #hemisphere title
    hemisphere_dict['title'] = browser.find_by_css('h2.title').text
    
    hemisphere_image_links.append(hemisphere_dict)
    
    browser.back()

In [83]:
#checking the list and dict
hemisphere_image_links

[{'img_url': 'https://www.marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://www.marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://www.marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://www.marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [90]:
def scrape_all():
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=True)
    data = {
        "news_title": news_title,
        "news_p": news_p,
        "featured_image": featured_image_url(browser),
        "facts": mars_facts(),
        "hemispheres": hemispheres(browser),
    }
    browser.quit()
    return(data)

In [91]:
def featured_image_url():
    url="https://spaceimages-mars.com/"
    html = browser.visit(url)
    #locate the button and click it
    image_element = browser.find_by_tag("button")[1]
    image_element.click()
    #Parse with beautiful soup
    html = browser.html
    img_soup = BeautifulSoup(html, 'html.parser')
    #Select image src and add it to the url
    image_anchor = img_soup.find("img", class_="fancybox-image").get('src')
    featured_image_url = url + image_anchor
    return featured_image_url